In [1]:
import numpy as np
import pandas as pd

from utils.drl_backtest import (
    generate_sliding_windows,
    train_agents,
    select_best_agent,
    backtest_all_agents
)
from utils.portfolio_env import PortfolioEnv
from utils.drl_agent import DRLAgent

In [2]:
# --- Load your data ---
returns_df = pd.read_parquet("../data/returns.parquet")
prices_df = pd.read_parquet("../data/prices.parquet")
vol_df = pd.read_parquet("../data/vola.parquet")

In [3]:
# --- Pick the first sliding window: 2006–2012 (train), 2011 (val), 2012 (test) ---
window = list(generate_sliding_windows(2006, 2021))[0]
print("Window:", window)

Window: {'train': ('2006-01-01', '2011-01-01'), 'val': ('2011-01-01', '2012-01-01'), 'test': ('2012-01-01', '2013-01-01')}


In [4]:
# --- Train 5 agents ---
agents = train_agents(DRLAgent, PortfolioEnv, returns_df, prices_df, vol_df, window, n_seeds=5)

# --- Select best agent based on Sharpe in validation year ---
best_agent = select_best_agent(agents, PortfolioEnv, returns_df, prices_df, vol_df, window)

# --- Backtest all 5 agents on the test year (2012) ---
all_metrics = backtest_all_agents(agents, PortfolioEnv, returns_df, prices_df, vol_df, window)

# --- Summarize Results ---
def summarize_metrics(metrics_list):
    summary = {}
    keys = metrics_list[0].keys()
    for key in keys:
        values = [m[key] for m in metrics_list]
        if isinstance(values[0], (int, float)):
            summary[key] = {
                "mean": np.mean(values),
                "std": np.std(values)
            }
    return summary

summary = summarize_metrics(all_metrics)

print("\n📊 Aggregated Evaluation Metrics (Mean ± Std over 5 agents):")
for k, v in summary.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

Output()

Output()

Output()

Output()

Output()


Evaluation Summary:
Final Portfolio Value: $101,892.30
Average Reward: -0.8836

Performance Metrics:
Annual return: 0.0162
Cumulative returns: 0.0123
Annual volatility: 0.2167
Sharpe ratio: 0.0906
Calmar ratio: 0.1811
Stability: 0.8219
Max drawdown: -0.0895
Omega ratio: 1.0152
Sortino ratio: 0.1382
Skew: 0.0820
Kurtosis: 1.1467
Tail ratio: 0.9723
Daily value at risk: -0.0241
Portfolio turnover: 0.0103
Average Reward: -0.8836


TypeError: Pickling an AuthenticationString object is disallowed for security reasons